In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
import requests
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool

THIẾT LẬP CHỐNG CHẶN BOT CỦA WEBSITE

In [5]:
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

ĐỌC FILE CSV ĐỂ LẤY RA CÁC URL BÀI VIẾT

In [6]:
df = pd.read_csv('../URL/urls_all_page_thang_6_301_400_2.csv')
urls = df['URL'].tolist()

XỬ LÝ LẤY PAGE_SOURCE CỦA POST 

SELLER INFOMATION

In [7]:
def Sales_Info(page_source):
    seller_info = page_source.find('div', class_ = 'SellerInfo_sellerWrapper__r4S9i')

    seller_name = seller_info.find('div', class_ = 'SellerInfo_nameDiv__rWqQB').text.strip()
    seller_role = seller_info.find('div', class_ = 'd4deiyo').text.strip()
    
    return seller_name, seller_role

THÔNG TIN CHUNG VỀ BẤT ĐỘNG SẢN

In [8]:
def RealEstate_Info(page_source, driver):
    RealEstate_Info = page_source.find('div', class_ = 'cd9gm5n')
    
    details = driver.find_elements(By.CLASS_NAME, 'brnpcl3')

    items = driver.find_elements(By.CSS_SELECTOR, 'div.i1qen30x.r9vw5if')

    phong_ngu = 'Không rõ'
    huong = 'Không rõ'
    loai_nha = 'Không rõ'
    for item in items:
        try:
            lines = item.text.strip().split('\n')  # Tách từng dòng riêng biệt
            for line in lines:
                line = line.strip()
                if 'PN' in line:
                    phong_ngu = line  # Không cần lấy lại từ <span>, vì line đã là nội dung chuẩn
                elif 'Hướng' in line:
                    huong = line
                elif 'Nhà' in line or 'Chung cư' in line or 'Căn hộ' in line or 'Penhouse' in line or 'Đất' in line or 'Tập thể' in line:
                    loai_nha = line
        except Exception as e:
            continue



    RE_Money = RealEstate_Info.find('b', class_ = 'pyhk1dv').text.strip()
    RE_Money_Per_SqM = details[0].text.strip()  # Giá tiền trên m2
    RE_Area = details[1].text.strip()   # Diện tích đất

    RE_Location = RealEstate_Info.find('span', class_ = 'bwq0cbs flex-1').text.strip()
    
    return phong_ngu, huong, loai_nha, RE_Money, RE_Money_Per_SqM, RE_Area, RE_Location


THÔNG TIN CHI TIẾT CỦA BẤT ĐỘNG SẢN

In [9]:
def RealEstate_Detail(driver):
    # Lấy tất cả các thẻ div thông tin
    items = driver.find_elements(By.CSS_SELECTOR, "div.col-xs-6.abzctes")

    giay_to_phap_ly = "Không rõ"
    so_phong_ve_sinh = "Không rõ"
    tinh_trang_noi_that = "Không rõ"
    so_tang = "Không rõ"

    for item in items:
        try:
            label = item.text.strip()
            if "Giấy tờ pháp lý" in label:
                # Tìm strong hoặc thẻ chứa giá trị trong cùng item
                value_el = item.find_element(By.TAG_NAME, "strong")
                giay_to_phap_ly = value_el.text.strip()
            elif "Số phòng vệ sinh" in label:
                value_el = item.find_element(By.TAG_NAME, "strong")
                so_phong_ve_sinh = value_el.text.strip()
            elif "Tình trạng nội thất" in label:
                value_el = item.find_element(By.TAG_NAME, "strong")
                tinh_trang_noi_that = value_el.text.strip()
            elif "Tổng số tầng" in label:
                value_el = item.find_element(By.TAG_NAME, "strong")
                so_tang = value_el.text.strip()           
        except:
            continue

    return giay_to_phap_ly, so_phong_ve_sinh, tinh_trang_noi_that, so_tang


- HÀM XỬ LÝ THAO TÁC NHẤN VÀO GIỮA MÀN HÌNH CHO LẦN ĐẦU
- CUỘN XUỐNG VÀ NHẤN VÀO NÚT XEM THÊM 
- LẤY CÁC THÔNG TIN CHI TIẾT

In [10]:
def process_url(driver, url):
    print(f"\nĐang xử lý: {url}")
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )
    except:
        print(f"Không load được trang: {url}")
        return None

    try:
        actions = ActionChains(driver)
        actions.move_by_offset(10, 10).click().perform()
        sleep(2)
    except Exception as e:
        print(f"Không thể click offset: {e}")

    try:
        more_button = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[4]/div[1]/div/div[4]/div/div[2]/div/div[2]/div[2]/button')
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", more_button)
        more_button.click()

        # Đợi phần tử dữ liệu mới xuất hiện sau khi click
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".your-new-data-selector"))
        )

    except:
        print(f"Không tìm thấy hoặc không click được nút 'Xem thêm'")

    # Chỉ lấy HTML sau khi dữ liệu đã load xong
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        seller_name, seller_role = Sales_Info(soup)
    except:
        seller_name, seller_role = "Không rõ", "Không rõ"

    try:
        phong_ngu, huong, loai_nha, RE_Money, RE_Money_Per_SqM, RE_Area, RE_Location = RealEstate_Info(soup, driver)
    except:
        phong_ngu = huong = loai_nha = RE_Money = RE_Money_Per_SqM = RE_Area = RE_Location = "Không rõ"

    try:
        giay_to_phap_ly, so_phong_ve_sinh, tinh_trang_noi_that, so_tang = RealEstate_Detail(driver)
    except:
        giay_to_phap_ly = so_phong_ve_sinh = tinh_trang_noi_that = so_tang = "Không rõ"

    return [
        url,
        seller_name, seller_role,
        phong_ngu, huong, loai_nha,
        RE_Money, RE_Money_Per_SqM, RE_Area, RE_Location,
        giay_to_phap_ly, so_phong_ve_sinh, tinh_trang_noi_that, so_tang
    ]


In [11]:
def scrape_url_wrapper(args):

    url, index = args

    options = uc.ChromeOptions()
    #options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")

    driver = uc.Chrome(options=options)
    
    try:
        result = process_url(driver, url)
    except Exception as e:
        print(f"Lỗi xử lý {url}: {e}")
        result = [url] + ["Lỗi"] * 13
    finally:
        driver.quit()

    return result

In [12]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

lock = threading.Lock()

def scrape_threadsafe(args):
    # lock không bắt buộc, nhưng có thể dùng nếu bị race condition
    # với thao tác driver global hoặc ghi log
    # với undetected_chromedriver bạn có thể bỏ lock nếu mỗi thread có 1 driver riêng
    return scrape_url_wrapper(args)

if __name__ == "__main__":
    url_args = [(url, i) for i, url in enumerate(urls)]

    print("Bắt đầu chạy")

    results = []
    max_threads = 5 

    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = [executor.submit(scrape_threadsafe, arg) for arg in url_args]

        for i, future in enumerate(as_completed(futures)):
            try:
                result = future.result()
                results.append(result)
                print(f"Hoàn thành {i + 1}/{len(url_args)}")
            except Exception as e:
                print(f"Lỗi thread {i + 1}: {e}")
                results.append(["Lỗi"] * 14)

    print("✅ Hoàn tất đa luồng.")


Bắt đầu chạy

Đang xử lý: https://www.nhatot.com/mua-ban-nha-dat-quan-hai-ba-trung-ha-noi/125720794.htm
Lỗi thread 1: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\CEO\\appdata\\roaming\\undetected_chromedriver\\undetected\\chromedriver-win32\\chromedriver.exe' -> 'C:\\Users\\CEO\\appdata\\roaming\\undetected_chromedriver\\undetected_chromedriver.exe'

Đang xử lý: https://www.nhatot.com/mua-ban-nha-dat-quan-cau-giay-ha-noi/124519497.htm
Lỗi thread 2: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\CEO\\appdata\\roaming\\undetected_chromedriver\\undetected\\chromedriver-win32\\chromedriver.exe' -> 'C:\\Users\\CEO\\appdata\\roaming\\undetected_chromedriver\\undetected_chromedriver.exe'

Đang xử lý: https://www.nhatot.com/mua-ban-nha-dat-quan-cau-giay-ha-noi/125720304.htm
Lỗi thread 3: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\CEO\\appdata\\roaming\\undetected_ch

VÒNG FOR CHẠY CÁC HÀM BÊN TRÊN

In [13]:
# Lưu dữ liệu ra file CSV
columns = [
    "URL",
    "Người bán", "Loại người bán",
    "Số phòng ngủ", "Hướng", "Kiểu nhà",
    "Giá", "Giá mỗi m2", "Diện tích", "Địa chỉ",
    "Giấy tờ pháp lý", "Số phòng vệ sinh", "Tình trạng nội thất", "Số tầng"
]
df_output = pd.DataFrame(results, columns=columns)
df_output.to_csv('../Detail/thong_tin_bds_thang_6_301_400_2.csv', index=False, encoding='utf-8-sig')

print("Đã lưu dữ liệu vào file thong_tin_bds_thang_6_301_400_2.csv")

Đã lưu dữ liệu vào file thong_tin_bds_thang_6_301_400_2.csv


PHẦN CHƯA DÙNG TỚI

In [14]:
# driver = uc.Chrome(options=options)
# actions = ActionChains(driver)

# results = []

# for i, url in enumerate(urls):
#     results.append(process_url(driver, url, first_time=(i == 0)))

In [15]:
# driver = uc.Chrome(options=options)
# actions = ActionChains(driver)
# sleep(1)

# results = []

# for i, url in enumerate(urls):
#     print(f"\nĐang xử lý: {url}")
#     driver.get(url)
#     sleep(2)

#     # Click vào vị trí (0, 0) hoặc giữa màn hình
#     if i == 0:
#         try:
#             actions.move_by_offset(10, 10).click().perform()
#             sleep(3)
#         except Exception as e:
#             print(f"Không thể click offset: {e}")

#     # Click vào nút "Xem thêm" để tải thêm nội dung và chờ 5 giây để đầy đủ nội dung
#     try:
#         more_field = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[4]/div[1]/div/div[4]/div/div[2]/div/div[2]/div[2]/button')
#         driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", more_field)
#         sleep(2)
#         more_field.click()
#         sleep(1)
#     except Exception as e:
#         print(f"Không tìm thấy hoặc không click được nút 'Xem thêm': {e}")

#     page_source = BeautifulSoup(driver.page_source, 'html.parser')

#     try:
#         seller_name, seller_role = Sales_Info(page_source)
#     except:
#         seller_name, seller_role = "Không rõ", "Không rõ"

#     try:
#         phong_ngu, huong, loai_nha, RE_Money, RE_Money_Per_SqM, RE_Area, RE_Location = RealEstate_Info(page_source, driver)
#     except:
#         phong_ngu = huong = loai_nha = RE_Money = RE_Money_Per_SqM = RE_Area = RE_Location = "Không rõ"

#     try:
#         giay_to_phap_ly, so_phong_ve_sinh, tinh_trang_noi_that, so_tang = RealEstate_Detail(driver)
#     except:
#         giay_to_phap_ly = so_phong_ve_sinh = tinh_trang_noi_that = so_tang = "Không rõ"

#     # Thêm dữ liệu vào list
#     results.append([
#         url,
#         seller_name, seller_role,
#         phong_ngu, huong, loai_nha,
#         RE_Money, RE_Money_Per_SqM, RE_Area, RE_Location,
#         giay_to_phap_ly, so_phong_ve_sinh, tinh_trang_noi_that, so_tang
#     ])